In [ ]:
import sys
import re
import string
import pickle
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
import joblib

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

import tweepy
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sb
import matplotlib.pyplot as plt
import wordcloud
from wordcloud import WordCloud, STOPWORDS
from collections import Counter

# EDA

In [ ]:
df = pd.read_csv("../input/mbti-type/mbti_1.csv")
df.head()

In [ ]:
df.isnull().any()#Checking if there are any missing or null values present in the dataset.


In [ ]:
df.shape #The shape of the dataset

In [ ]:
df.info() #information about dataset

In [ ]:
types = np.unique(np.array(df['type']))
types

In [ ]:
print(df.type.value_counts())
df.type.hist(xrot=90)
plt.show()

In [ ]:
graph = df['type'].value_counts()
plt.figure(figsize=(15,6))
sb.barplot(graph.index, graph.values, alpha=1)
plt.xlabel('Personality types', fontsize=15)
plt.ylabel('No. of posts', fontsize=15)
plt.show()

Above we can see that there is great unbalance in Introvert/Extrovert and Intuition/Sensing pairs. Whereas Feeling/Thinking and Perception/Judgment pairs are quite balanced. Although I have created trained models for each pair. Only last 2 pairs are somewhat reliable in predicting MBTI type. So it is not advised to depend on first 2 pairs i.e. IE and NS pairs.

In [ ]:
# Swarm Plot
df1 = df.copy()
#this function counts the no of words in each post of a user
def var_row(row):
    l = []
    for i in row.split('|||'):
        l.append(len(i.split()))
    return np.var(l)

#this function counts the no of words per post out of the total 50 posts in the whole row
df1['words_per_comment'] = df1['posts'].apply(lambda x: len(x.split())/50)
df1['variance_of_word_counts'] = df1['posts'].apply(lambda x: var_row(x))

plt.figure(figsize=(15,10))
sb.swarmplot("type", "words_per_comment", data=df1)

In [ ]:
#Plotting WordCloud.

#Finding the most common words in all posts.
words = list(df1["posts"].apply(lambda x: x.split()))
words = [x for y in words for x in y]
Counter(words).most_common(40)
wc = wordcloud.WordCloud(width=1200, height=500, 
                         collocations=False, background_color="white", 
                         colormap="tab20b").generate(" ".join(words))

# collocations to False  is set to ensure that the word cloud doesn't appear as if it contains any duplicate words
plt.figure(figsize=(25,10))
# generate word cloud, interpolation 
plt.imshow(wc, interpolation='bilinear')
_ = plt.axis("off")



In [ ]:
fig, ax = plt.subplots(len(df1['type'].unique()), sharex=True, figsize=(15,len(df1['type'].unique())))
k = 0
for i in df1['type'].unique():
    df_4 = df[df['type'] == i]
    wordcloud = WordCloud(max_words=1628,relative_scaling=1,normalize_plurals=False).generate(df_4['posts'].to_string())
    plt.subplot(4,4,k+1)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(i)
    ax[k].axis("off")
    k+=1

# Preprocessing

I will add one column for each MBTI characteristic pair, since we will be training independent classifier model for each pair independently. The reason for this is because of imbalance present in our dataset as seen in the EDA section.

In [ ]:


df['ie'] = df.type
df['ns'] = df.type
df['ft'] = df.type
df['pj'] = df.type

for i, t in enumerate(df.type):
    if 'I' in t:
        df.ie[i] = 'I'
    elif 'E' in t:
        df.ie[i] = 'E'
        
    if 'N' in t:
        df.ns[i] = 'N'
    elif 'S' in t:
        df.ns[i] = 'S'
        
    if 'F' in t:
        df.ft[i] = 'F'
    elif 'T' in t:
        df.ft[i] = 'T'
        
    if 'P' in t:
        df.pj[i] = 'P'
    elif 'J' in t:
        df.pj[i] = 'J'


posts = df.posts.values
yIE = df.ie.values
yNS = df.ns.values
yFT = df.ft.values
yPJ = df.pj.values
y = df.type

In [ ]:
print(posts.shape)

In [ ]:
print(df.head(5))

In [ ]:
print(df.ie.value_counts(), end='\n\n')
print(df.ns.value_counts(), end='\n\n')
print(df.ft.value_counts(), end='\n\n')
print(df.pj.value_counts(), end='\n\n')

df.ie.hist(); plt.show()
df.ns.hist(); plt.show()
df.ft.hist(); plt.show()
df.pj.hist(); plt.show()

How many Introvert posts are present v/s how many Extrovert posts are presnt, out of all the given entries in our labelled Kaggle dataset. This is done in order to extplore the dataset for all the individual Personality Indices of MBTI

Counting No. of posts in one class / Total no. of posts in the other class


In [ ]:
print ("Introversion (I) /  Extroversion (E):\t", df['ie'].value_counts()['I'], " / ", df['ie'].value_counts()['E'])
print ("Intuition (N) / Sensing (S):\t\t", df['ns'].value_counts()['N'], " / ", df['ns'].value_counts()['S'])
print ("Thinking (T) / Feeling (F):\t\t", df['ft'].value_counts()['F'], " / ", df['ft'].value_counts()['T'])
print ("Judging (J) / Perceiving (P):\t\t", df['pj'].value_counts()['P'], " / ", df['pj'].value_counts()['J'])

In [ ]:
#regular expressions for tokenization
regexes = [
    #urls
    #r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',
    
    #html
    #r'<[^>]+>',
    
    #punctuation
    r'(?:(\w+)\'s)',
    
    r'(?:\s(\w+)\.+\s)',
    r'(?:\s(\w+),+\s)',
    r'(?:\s(\w+)\?+\s)',
    r'(?:\s(\w+)!+\s)',
    
    r'(?:\'+(\w+)\'+)',
    r'(?:"+(\w+)"+)',
    r'(?:\[+(\w+)\]+)',
    r'(?:{+(\w+)}+)',
    r'(?:\(+(\w+))',
    r'(?:(\w+)\)+)',

    #words containing numbers & special characters & punctuation
    r'(?:(?:(?:[a-zA-Z])*(?:[0-9!"#$%&\'()*+,\-./:;<=>?@\[\\\]^_`{|}~])+(?:[a-zA-Z])*)+)',
    
    #pure words
    r'([a-zA-Z]+)',
    
    #numbers
    #r'(?:(?:\d+,?)+(?:\.?\d+)?)',

    #emoticons
    #r"""(?:[:=;][oO\-]?[D\)\]\(\]/\\OpP])""",

    #other words
    #r'(?:[\w_]+)',

    #anything else
    #r'(?:\S)'
]

#compiling regular expression
regex = re.compile(r'(?:'+'|'.join(regexes)+')', re.VERBOSE | re.IGNORECASE)


In [ ]:
def preprocess(documents):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    #fetching list of stopwords
    punctuation = list(string.punctuation)
    swords = stopwords.words('english') + ['amp'] + ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'january', 'feburary', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december',  'mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun',  'jan', 'feb', 'mar', 'apr', 'may', 'jun' 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'tommorow', 'today', 'yesterday'] + ['mr', 'mrs']


    processed_documents = []
    for i,document in enumerate(documents):
        print('{0}/{1}'.format(i+1, len(documents)))
        
        #tokenization
        tokens = regex.findall(document)

        #skipping useless tokens
        t_regex = re.compile(r"[^a-zA-Z]")
        document = []
        
        for token in tokens:
            token = np.array(token)
            token = np.unique(token[token != ''])
            
            if len(token) > 0:
                token = token[0].lower()
            else:
                continue
                
            if re.search(t_regex, token) == None and token not in swords:
                token = lemmatizer.lemmatize(token)
                document.append(token)
                
        document = ' '.join(document)

        #skipping
        if len(document) >= 0:
            processed_documents.append(document)

    print()
    return np.array(processed_documents)

In [ ]:
%%time
posts = preprocess(posts)

In [ ]:
print(posts[0])

# Processing

In [ ]:
%%time

#TF-IDF representation
# creating document frequency matrix
cv = CountVectorizer().fit(posts)
X = cv.transform(posts)

In [ ]:
tf = TfidfTransformer()

In [ ]:
X_tf=  tf.fit_transform(X).toarray()

In [ ]:
print(X)

In [ ]:
posts.shape, X.shape, X_tf.shape, yIE.shape, yNS.shape # verifying that the shapes match

In [ ]:


print("X: 1st posts in tf-idf representation\n%s" % X_tf[0])


# Training

In [ ]:
# splitting dataset into training and testing dataset
xIETrain, xIETest, yIETrain, yIETest = train_test_split(X, yIE)
xNSTrain, xNSTest, yNSTrain, yNSTest = train_test_split(X, yNS)
xFTTrain, xFTTest, yFTTrain, yFTTest = train_test_split(X, yFT)
xPJTrain, xPJTest, yPJTrain, yPJTest = train_test_split(X, yPJ)
xTrain, xTest, yTrain, yTest = train_test_split(X, y)

In [ ]:
model = MultinomialNB().fit(xTrain, yTrain)
ieModel = MultinomialNB().fit(xIETrain, yIETrain)
nsModel = MultinomialNB().fit(xNSTrain, yNSTrain)
ftModel = MultinomialNB().fit(xFTTrain, yFTTrain)
pjModel = MultinomialNB().fit(xPJTrain, yPJTrain)

# Testing - Accuracy, Recall, f1

In [ ]:
print(model.score(xTest, yTest))
print(ieModel.score(xIETest, yIETest))
print(nsModel.score(xNSTest, yNSTest))
print(ftModel.score(xFTTest, yFTTest))
print(pjModel.score(xPJTest, yPJTest))

As we can see above the first model which tries to predict at at once performs poorly. But the separate model which predicts i or e, n or s, f or t, p or j does good

In [ ]:
# Let's look at the recall score

#minority classes
print('MINORITY CLASSES:')
print(recall_score(yIETest, ieModel.predict(xIETest), pos_label='E'))
print(recall_score(yNSTest, nsModel.predict(xNSTest), pos_label='S'))
print(recall_score(yFTTest, ftModel.predict(xFTTest), pos_label='T'))
print(recall_score(yPJTest, pjModel.predict(xPJTest), pos_label='J'), end='\n\n')

#majority classes
print('MAJORITY CLASSES:')
print(recall_score(yIETest, ieModel.predict(xIETest), pos_label='I'))
print(recall_score(yNSTest, nsModel.predict(xNSTest), pos_label='N'))
print(recall_score(yFTTest, ftModel.predict(xFTTest), pos_label='F'))
print(recall_score(yPJTest, pjModel.predict(xPJTest), pos_label='P'))

In [ ]:
# Let's look at the f1 score

#minority classes
print('MINORITY CLASSES:')
print(f1_score(yIETest, ieModel.predict(xIETest), pos_label='E'))
print(f1_score(yNSTest, nsModel.predict(xNSTest), pos_label='S'))
print(f1_score(yFTTest, ftModel.predict(xFTTest), pos_label='T'))
print(f1_score(yPJTest, pjModel.predict(xPJTest), pos_label='J'), end='\n\n')

#majority classes
print('MAJORITY CLASSES:')
print(f1_score(yIETest, ieModel.predict(xIETest), pos_label='I'))
print(f1_score(yNSTest, nsModel.predict(xNSTest), pos_label='N'))
print(f1_score(yFTTest, ftModel.predict(xFTTest), pos_label='F'))
print(f1_score(yPJTest, pjModel.predict(xPJTest), pos_label='P'))

To get actual performance of our models, I am using KFold cross validation with k=10 to get actual performance. These values will be pickled along with models, so that these can be used in scripts to allow users to see performance and reliability of each model corresponding to their characteristic pair.

In [ ]:
scores = []

scores.append(cross_val_score(estimator=model, cv=10, X=X, y=y, scoring='accuracy'))
scores.append(cross_val_score(estimator=ieModel, cv=10, X=X, y=LabelEncoder().fit_transform(yIE), scoring='recall'))
scores.append(cross_val_score(estimator=nsModel, cv=10, X=X, y=LabelEncoder().fit_transform(yNS), scoring='recall'))
scores.append(cross_val_score(estimator=ftModel, cv=10, X=X, y=LabelEncoder().fit_transform(yFT), scoring='recall'))
scores.append(cross_val_score(estimator=pjModel, cv=10, X=X, y=LabelEncoder().fit_transform(yPJ), scoring='recall'))

#prining mean and standard deviations for each model
for score in scores:
    print(score.mean())
    print(score.std(), end='\n\n')

# Prediction using text

In [ ]:
my_posts =''' Enter your text here '''
my_posts = [my_posts]
mydata = pd.DataFrame(data={'type': ['xxxx'], 'posts': [my_posts]})

document = cv.transform(my_posts)

print(document)

In [ ]:
print(ieModel.predict(document))
print(nsModel.predict(document))
print(ftModel.predict(document))
print(pjModel.predict(document))

We can also predict a person's personality by looking at his twitter tweets

# Prediction using tweets

In [ ]:
CONSUMER_KEY        = 'XXX'
CONSUMER_SECRET     = 'XXX'
ACCESS_TOKEN        = 'XXX'
ACCESS_TOKEN_SECRET = 'XXX'

AUTH = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
AUTH.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(AUTH)

In [ ]:
#getting user tweets

tweets = api.user_timeline('username', count=200)
tweets = [tweet.text for tweet in tweets]

In [ ]:
document = cv.transform([' '.join(tweets)])


In [ ]:
print(ieModel.predict(document))
print(nsModel.predict(document))
print(ftModel.predict(document))
print(pjModel.predict(document))